# Human Longevity - base on Diet, Social, and Economic factors

## Import Modules, Data, and Dicts

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Paths
#If using Google Colab
path = '/content/drive/MyDrive/SIADS_591-592/milestone_1_team'
#If using local Jupyter Notebook
#path = '.'

In [3]:
import sys
sys.path.insert(1, path + '/dicts')
import country_codes as cc

In [4]:
#Life Expectancy
life_df = pd.read_csv(path + '/data/life_expectancy/Life Expectancy Data.csv')
#life_df_dont_use = pd.read_excel('./data/life_expectancy/Life Expectancy at Birth.xlsx')

#Diet
fat_df = pd.read_csv(path + '/data/diet/Fat_Supply_Quantity_Data.csv')
cal_df = pd.read_csv(path + '/data/diet/Food_Supply_kcal_Data.csv')
meat_df = pd.read_csv(path + '/data/diet/meat_consumption_worldwide.csv')

#Social
trust_df = pd.read_csv(path + '/data/social/cross-country-variation-in-trust.csv')
friend_df = pd.read_csv(path + '/data/social/learn_info_friends.csv')
friend_imp_df = pd.read_csv(path + '/data/social/how_important_are_friends.csv')

#Final Analytical dataset
final_df = pd.read_csv(path + '/data/analytical_dataset_milestone_1.csv')

## Cleaning

Generic Cleaning Tasks

In [5]:
def preclean(df):
    
    #Strip extra whitespace from column names and set lowercase
    df.rename(columns=lambda x: ('_'.join(x.split())).lower(), inplace=True)
    
    return df
    
def since_2010(df, year_col, groupby_cols):
    df = df[pd.to_numeric(df[year_col]) >= 2010]
    df = df.groupby(by=groupby_cols).mean()
    
    return df

def postclean(df, drop_cols):
    
    #Drop specified columns
    df.drop(labels=drop_cols, axis=1, inplace=True)

    #Drop all rows that have a NaN value (this comes after the grouping so should be less severe)
    df.dropna(how='any', inplace=True)
    
    #Sort by Index
    df.sort_index(inplace=True)
    
    return df
    
life_wdf = postclean(since_2010(preclean(life_df), 'year', ['country', 'status']), ['year'])
trust_wdf = postclean(since_2010(preclean(trust_df), 'year', ['entity']), ['year'])
meat_wdf = postclean(since_2010(preclean(meat_df), 'time', ['location','subject','measure']), ['time'])

In [6]:
final_df.head()

,Unnamed: 0,code,country,life expectancy,adult mortality,infant deaths,alcohol,percentage expenditure,measles,under-five deaths,polio,total expenditure,diphtheria,hiv/aids,alcoholic beverages,animal products,animal fats,"aquatic products, other",cereals - excluding beer,eggs,"fish, seafood",fruits - excluding wine,meat,milk - excluding butter,miscellaneous,offals,oilcrops,pulses,spices,starchy roots,stimulants,sugar crops,sugar & sweeteners,treenuts,vegetal products,vegetable oils,vegetables,obesity,population,per_capita_gdp_ppp,iso_3
0,0,AF,Afghanistan,60.383333,271.333333,67.666667,0.010000,63.830523,1644.166667,91.666667,54.500000,8.343333,65.333333,0.100000,0.0000,4.7774,0.8504,0.0,37.1186,0.1501,0.0000,1.4757,1.2006,2.4512,0.0250,0.1251,0.1751,0.5003,0.1001,0.3252,0.0750,0.0,2.2261,0.1251,45.2476,2.3012,0.7504,4.5,38928000.0,1976.0,AFG
1,1,AL,Albania,77.033333,71.833333,0.166667,4.943333,352.654915,7.833333,0.833333,98.833333,5.696667,98.833333,0.100000,0.9120,16.0930,1.0591,0.0,16.2107,0.8091,0.1471,3.8982,3.8688,9.9441,0.0588,0.2648,1.0886,0.8091,0.0000,1.2651,0.2501,0.0,3.4422,0.3972,33.9070,2.8244,2.7508,22.3,2838000.0,12943.0,ALB
2,2,DZ,Algeria,75.166667,81.666667,21.000000,0.442000,349.055629,53.500000,24.000000,95.000000,6.176000,95.000000,0.100000,0.0896,6.0326,0.1941,0.0,25.0112,0.4181,0.1195,3.1805,1.2543,3.9869,0.1045,0.0597,0.2688,1.0900,0.1195,1.9262,0.1493,0.0,3.9869,0.2240,43.9749,5.7638,2.0457,26.6,44357000.0,15293.0,DZA
3,3,AO,Angola,51.966667,353.666667,71.166667,8.106000,124.598640,4572.833333,108.333333,61.833333,3.528000,71.333333,2.300000,1.9388,4.6927,0.2644,0.0,18.3521,0.0441,0.8372,2.3133,2.9302,0.5067,0.0661,0.1102,1.0795,1.4981,0.0000,12.6239,0.0441,0.0,2.7539,0.0000,45.3184,4.2741,0.3525,6.8,32522000.0,6658.0,AGO
4,4,AG,Antigua and Barbuda,75.983333,114.166667,0.000000,8.200000,1727.582040,0.000000,0.000000,95.833333,5.508000,98.666667,0.166667,2.3041,15.3672,1.5429,0.0,13.7215,0.2057,1.7280,3.6824,7.0356,4.6904,0.3086,0.1646,0.5966,0.4526,0.3497,0.8434,0.4937,0.0,5.8218,0.0823,34.6225,4.6904,1.2960,19.1,98000.0,23522.0,ATG


Specific Cleaning Tasks

In [7]:
#life

#Demoting Indices to columns
life_cdf = life_wdf.reset_index(level=['country','status'])

#Renaming values
life_cdf.country.replace('Cabo Verde','Cape Verde', inplace=True)

#status column to binary
life_cdf['developed'] = life_cdf.status.map(lambda x: 1 if x == 'Developed' else 0)

#Setting country and code as index
life_cdf.set_index(['country'], inplace=True)

#dropping unnecessary columns
life_cdf.drop(['status'],axis='columns', inplace=True)

#meat

#Demoting Indices to columns
meat_cdf = meat_wdf.reset_index(level=['location','subject','measure'])

#Creating new country code column
meat_cdf['code'] = meat_cdf['location'].map(cc.iso3_iso2)

#Creating new country column
meat_cdf['country'] = meat_cdf['code'].map(cc.iso2_country)

#Renaming values
meat_cdf.country.replace('Russia','Russian Federation', inplace=True)

#Dropping iso3 codes without a country
meat_cdf = meat_cdf[meat_cdf['country'].notna()]

#Selecting for KG per capita annually
meat_cdf = meat_cdf[meat_cdf.measure == 'KG_CAP']

#Setting country as index
meat_cdf.set_index(['country'], inplace=True)

#dropping unnecessary columns
meat_cdf.drop(['location','measure','code'],axis='columns', inplace=True)

#Pivot to put 'subject' (meat types) as columns
meat_cdf = meat_cdf.pivot(index=None, columns='subject',values='value')

meat_cdf = preclean(meat_cdf)

#trust

#Demoting Indices to columns
trust_cdf = trust_wdf.reset_index()

#renaming column
trust_cdf.columns = ['country','trust_index']

#Renaming values
trust_cdf.country.replace('Russia','Russian Federation', inplace=True)

#Setting country as index
trust_cdf.set_index(['country'], inplace=True)

#Final DF

final_cdf = final_df.copy(deep=True)
final_cdf.drop(['Unnamed: 0', 'code','iso_3'],axis='columns', inplace=True)
final_cdf.set_index(['country'], inplace=True)

In [8]:
trust_wdf.head()

,trust_(falk_et_al._(2018)_gps)
entity,
Afghanistan,0.315964
Algeria,-0.177371
Argentina,-0.206733
Australia,0.298973
Austria,0.154628


In [9]:
print(len(set(life_cdf.index.unique())))
print(len(set(meat_cdf.index.unique())))
print(len(set(trust_cdf.index.unique())))
#print(sorted(set(meat_cdf.index.unique()) - set(life_cdf.index.unique())))
#print(sorted(set(life_cdf.index.unique()) - set(meat_cdf.index.unique())))
print(sorted(set(life_cdf.index.unique()) - set(trust_cdf.index.unique())))
print(sorted(set(trust_cdf.index.unique()) - set(life_cdf.index.unique())))

132
41
76
['Albania', 'Angola', 'Armenia', 'Azerbaijan', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cape Verde', 'Central African Republic', 'Chad', 'Comoros', 'Cyprus', 'Djibouti', 'Dominican Republic', 'Ecuador', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Fiji', 'Gabon', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Honduras', 'Ireland', 'Jamaica', 'Kiribati', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Luxembourg', 'Madagascar', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Mauritania', 'Mauritius', 'Mongolia', 'Montenegro', 'Mozambique', 'Myanmar', 'Namibia', 'Nepal', 'Niger', 'Panama', 'Papua New Guinea', 'Paraguay', 'Samoa', 'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Solomon Islands', 'Swaziland', 'Tajikistan', 'Timor-Leste', 'Togo', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkmenistan', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Zambia']
['Bolivia', 'Czechia', 'Egypt', 'Finland', 'Hungary', 'Iran', 'Japan

## Joining

In [10]:
#Trust is not that highly correlated with Life Expectancy
lifetrust_df = life_cdf.join(trust_cdf, how='inner')

#Meat consumption and life expectancy
lifemeat_df = life_cdf.join(meat_cdf, how='inner')


In [11]:
#Reducing to important columns for pairplot
life_rdf = life_cdf[['life_expectancy','alcohol', 'percentage_expenditure', 'bmi', 
                    'gdp', 'schooling', 'income_composition_of_resources']]
lifetrust_rdf = lifetrust_df[['life_expectancy','gdp','trust_index']]
lifemeat_rdf = lifemeat_df[['life_expectancy','gdp','bmi','beef','pig','poultry','sheep']]

## Analysis

In [12]:
print(len(final_cdf[final_cdf.per_capita_gdp_ppp < 5000]))
print(len(final_cdf[(final_cdf.per_capita_gdp_ppp > 5000) & (final_cdf.per_capita_gdp_ppp < 14000)]))
print(len(final_cdf[(final_cdf.per_capita_gdp_ppp > 14000) & (final_cdf.per_capita_gdp_ppp < 30000)]))
print(len(final_cdf[final_cdf.per_capita_gdp_ppp > 30000]))

42
41
40
37


In [14]:
def corr_one(inpdf, attr, rowname):
  df = inpdf.corr(method='spearman')
  df = df.loc[[attr]]
  df.rename(mapper={attr:rowname}, axis=0, inplace=True)
  return df

nofilter = corr_one(final_cdf, 'life expectancy', 'all')
poor = corr_one(final_cdf[final_cdf.per_capita_gdp_ppp < 5000], 'life expectancy', 'poor')
middling = corr_one(final_cdf[(final_cdf.per_capita_gdp_ppp > 5000) & (final_cdf.per_capita_gdp_ppp < 14000)], 'life expectancy', 'middling')
wealthy = corr_one(final_cdf[(final_cdf.per_capita_gdp_ppp > 14000) & (final_cdf.per_capita_gdp_ppp < 30000)], 'life expectancy', 'wealthy')
rich = corr_one(final_cdf[final_cdf.per_capita_gdp_ppp > 30000], 'life expectancy', 'rich')

together = pd.concat([nofilter, poor, middling, wealthy, rich])
together.to_csv(path_or_buf=path + '/WealthBins.csv')
together.head()

,life expectancy,adult mortality,infant deaths,alcohol,percentage expenditure,measles,under-five deaths,polio,total expenditure,diphtheria,hiv/aids,alcoholic beverages,animal products,animal fats,"aquatic products, other",cereals - excluding beer,eggs,"fish, seafood",fruits - excluding wine,meat,milk - excluding butter,miscellaneous,offals,oilcrops,pulses,spices,starchy roots,stimulants,sugar crops,sugar & sweeteners,treenuts,vegetal products,vegetable oils,vegetables,obesity,population,per_capita_gdp_ppp
all,1.0,-0.858391,-0.604953,0.565880,0.601536,-0.276415,-0.621653,0.538284,0.406538,0.539733,-0.768261,0.448899,0.763241,0.671721,0.140590,-0.555569,0.733648,0.286292,0.219935,0.618026,0.615610,0.107690,0.069238,-0.266558,-0.405113,0.152227,-0.370088,0.608490,-0.081634,0.509690,0.558236,-0.762964,0.255284,0.473518,0.601247,-0.074464,0.853465
poor,1.0,-0.793080,-0.328320,-0.245259,0.165200,-0.235189,-0.337968,0.431153,-0.051943,0.361780,-0.785718,-0.231945,0.195535,0.148535,NaN,-0.007131,0.383615,0.026297,0.047486,0.039788,0.120579,0.149937,-0.210940,-0.164418,-0.190843,0.147471,-0.307362,0.276094,0.340895,0.337345,-0.006334,-0.200397,-0.165471,0.119039,0.137958,-0.041408,0.471294
middling,1.0,-0.699303,-0.363529,0.122479,0.091800,-0.292010,-0.405911,0.442672,0.438153,0.421892,-0.602490,-0.048084,0.286063,0.177526,0.026726,-0.058711,0.391986,-0.039024,0.301220,0.043206,0.362021,0.104548,0.176481,-0.168641,-0.172852,0.027027,-0.232056,0.181359,0.002030,0.227526,0.321360,-0.285889,-0.175610,0.305749,0.421360,-0.167596,0.449477
wealthy,1.0,-0.779623,-0.214515,0.085651,0.368374,-0.213705,-0.243659,0.020656,0.595431,0.120750,-0.447382,-0.107416,0.284160,0.196538,0.104044,-0.194943,0.101037,0.258924,0.080585,0.211642,0.201229,-0.142361,-0.130828,0.032459,0.233313,0.222816,-0.330409,0.164548,-0.096032,0.313242,0.300296,-0.280407,-0.119893,0.123558,0.300422,0.006192,0.235377
rich,1.0,-0.222137,-0.031124,0.414464,0.384557,0.295387,0.028346,-0.009678,0.316064,-0.001010,-0.347839,0.316732,0.313100,0.340842,0.166963,-0.175578,-0.002134,0.109899,-0.071014,0.258328,0.162300,-0.327103,-0.041494,-0.079787,-0.466864,-0.373918,0.390871,0.076941,0.140505,-0.059277,0.236633,-0.313100,0.020628,0.142383,-0.203296,0.253468,0.206402


In [ ]:

final_rdf = final_cdf[['life expectancy', 'alcohol',
       'percentage expenditure', 'measles', 'polio',
       'total expenditure', 'diphtheria', 'hiv/aids', 'alcoholic beverages',
       'animal products', 'animal fats', 'aquatic products, other',
       'cereals - excluding beer', 'eggs', 'fish, seafood',
       'fruits - excluding wine', 'meat', 'milk - excluding butter',
        'offals', 'oilcrops', 'pulses', 'spices',
       'starchy roots', 'stimulants', 'sugar crops', 'sugar & sweeteners',
       'treenuts', 'vegetal products', 'vegetable oils', 'vegetables',
       'obesity', 'population', 'per_capita_gdp_ppp']]

print(final_cdf.columns)
print(len(final_cdf.columns))

In [ ]:
def disp_lr(linreg, feats, X_train, X_test, y_train, y_test):
  print(dict(zip(feats,['{0:0.3f}'.format(i) for i in linreg.coef_])))
  print('Number of features: {}'.format(len(feats)))
  print('linear model intercept (b): {:.3f}'.format(linreg.intercept_))
  print('R-squared score (training): {:.3f}'.format(linreg.score(X_train, y_train)))
  print('R-squared score (test): {:.3f}'.format(linreg.score(X_test, y_test)))
  return None

#Linear Regression
def build_lr(df, target, num_feats):

  #Entire set of feature + target data
  X = df.loc[:, df.columns != target]
  y = df.loc[:, target]

  #Creating linreg model
  linreg = LinearRegression()
  
  #finding the most important features to include
  rfe_mod = RFE(linreg, num_feats, step=1)
  feat_rank=rfe_mod.fit(X,y)
  msk = np.array(feat_rank.support_)
  feats = np.array(X.columns)
  myfeats = feats[msk].tolist()

  #Resetting feature data to only include the important features
  X = df.loc[:, myfeats]

  #Splitting and fitting
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)
  linreg.fit(X_train, y_train)

  #Feature coefficient values
  coeffs = dict(zip(feats,['{0:0.3f}'.format(i) for i in linreg.coef_]))

  #Show results
  disp_lr(linreg, myfeats, X_train, X_test, y_train, y_test)

  return linreg, coeffs, linreg.score(X_test, y_test)

'''
r_max = 0
best_lr = None
for i in range(1,len(final_rdf.columns) - 1):
  lr, coeffs, score = build_lr(final_rdf, 'life expectancy', i)
  if score > r_max:
    r_max = score
    best_lr = lr
print(r_max)
'''
lr, coeffs, score = build_lr(final_rdf, 'life expectancy', 29)
print(coeffs)
coeffs2 = {k : [v] for k, v in coeffs.items()}
pd.DataFrame.from_dict(coeffs2)

## Visualization

In [ ]:
plt.hist(final_rdf.per_capita_gdp_ppp)

In [ ]:
#sns.pairplot(final_rdf, height=3, aspect=1, kind='scatter', diag_kind='hist', markers = 'o', diag_kws = {'bins': 10})